In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Maranhão - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [6]:
data = pd.read_csv('2003_mo_model_input_MA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - Consumo de Cimento (t)
0,2003-1,23.368400,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.213662,0.260097,0.635570,4.315322e+07,2.636001e+06,6.034057,3.872495e+07,43.342
1,2003-2,23.167977,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.207986,0.258964,0.635997,4.320832e+07,2.636851e+06,6.036083,3.874405e+07,28.663
2,2003-3,23.563273,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.202310,0.257352,0.636424,4.326343e+07,2.637701e+06,6.038110,3.876316e+07,27.900
3,2003-4,24.008345,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.196633,0.255929,0.636851,4.331853e+07,2.638551e+06,6.040136,3.878226e+07,25.739
4,2003-5,24.366896,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.190957,0.254109,0.637278,4.337364e+07,2.639401e+06,6.042163,3.880136e+07,30.203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,31.724066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.520178,NaN,NaN,NaN,NaN,NaN,102.698
236,2022-9,31.679231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.518515,NaN,NaN,NaN,NaN,NaN,98.133
237,2022-10,31.524870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.516447,NaN,NaN,NaN,NaN,NaN,94.964
238,2022-11,31.512212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.513912,NaN,NaN,NaN,NaN,NaN,97.684


In [7]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,-1.197964,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.822290,-0.950106,-2.166110,-1.681167,-0.279527,-2.120335,-1.967581
1,-1.214041,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.825118,-0.970613,-2.129811,-1.663459,-0.262506,-2.086804,-1.938229
2,-1.182333,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.827947,-0.999751,-2.093513,-1.645750,-0.245485,-2.053272,-1.908877
3,-1.146631,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.830775,-1.025491,-2.057214,-1.628041,-0.228464,-2.019741,-1.879525
4,-1.117870,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.833603,-1.058403,-2.020915,-1.610333,-0.211443,-1.986210,-1.850174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.550687,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.189788,1.221276,1.327369,1.120107,-1.533048,0.771361,0.866519
188,-0.554816,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.185037,1.236987,1.321661,1.107960,-1.504452,0.745175,0.846505
189,-0.564631,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.180287,1.252847,1.315953,1.095812,-1.475855,0.718990,0.826491
190,-0.596444,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.175536,1.276241,1.310245,1.083664,-1.447259,0.692804,0.806477


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      42.076
1      30.183
2      40.701
3      39.783
4      38.903
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Maranhão - Consumo de Cimento (t), Length: 240, dtype: float64

In [9]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,-1.197964,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.822290,-0.950106,-2.166110,-1.681167,-0.279527,-2.120335,-1.967581
1,-1.214041,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.825118,-0.970613,-2.129811,-1.663459,-0.262506,-2.086804,-1.938229
2,-1.182333,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.827947,-0.999751,-2.093513,-1.645750,-0.245485,-2.053272,-1.908877
3,-1.146631,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.830775,-1.025491,-2.057214,-1.628041,-0.228464,-2.019741,-1.879525
4,-1.117870,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.833603,-1.058403,-2.020915,-1.610333,-0.211443,-1.986210,-1.850174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.796451,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.431469,0.944351,1.437372,1.197811,-1.616416,0.972153,1.061498
158,0.727411,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.418921,0.952621,1.436068,1.200726,-1.633804,0.973222,1.061628
159,0.657765,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.406373,0.960982,1.434764,1.203641,-1.651193,0.974290,1.061759
160,0.591941,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.393825,0.969363,1.433461,1.206556,-1.668581,0.975359,1.061889


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      42.076
1      30.183
2      40.701
3      39.783
4      38.903
        ...  
157    66.661
158    78.801
159    67.256
160    84.280
161    86.501
Name: Maranhão - Consumo de Cimento (t), Length: 162, dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
162,0.482008,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.368729,0.985437,1.430853,1.212386,-1.703358,0.977496,1.062150
163,0.429352,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.356181,0.993928,1.429550,1.215301,-1.720747,0.978564,1.062281
164,0.379500,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.343633,1.002369,1.428246,1.218215,-1.738136,0.979633,1.062412
165,0.330400,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.331085,1.010759,1.426942,1.221130,-1.755524,0.980702,1.062542
166,0.281480,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.318536,1.019096,1.425639,1.224045,-1.772913,0.981770,1.062673
167,0.221910,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.305988,1.027156,1.424335,1.226960,-1.790301,0.982839,1.062803
168,0.169785,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.293440,1.039065,1.423031,1.229875,-1.807690,0.983907,1.062934
169,0.100463,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.287574,1.053773,1.418389,1.227814,-1.801484,0.981470,1.058241
170,0.049693,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.281707,1.068604,1.413747,1.225752,-1.795278,0.979033,1.053548
171,-0.013951,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.275840,1.079870,1.409105,1.223691,-1.789073,0.976595,1.048855


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    88.780
163    86.195
164    84.121
165    88.187
166    79.514
167    74.061
168    64.818
169    44.254
170    52.169
171    53.399
172    45.364
173    67.103
174    61.062
175    73.445
176    69.471
177    83.004
178    77.332
179    58.591
180    70.660
181    50.796
182    47.834
183    45.327
184    54.032
185    60.657
186    67.250
187    70.773
188    67.010
189    81.128
190    81.737
191    76.841
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [13]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor//2):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
        pos = df.shape[0] - (i*div_factor + 5)
        positions_to_drop.append(pos)
        pos = df.shape[0] - (i*div_factor + 9)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 12)
    target,target_val = validation_splitter(train_target, 12)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [17]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1471347403, 2440018909, 1488588411, 3987304016, 1516793711, 198372032, 34181826, 709563659, 2682056259, 2271740860, 2983012698, 142052549, 556343551, 1699951117, 1215432422, 2376476659, 1316622454, 3874990641, 2806786100, 2323021787, 2565263292, 806036685, 805231936, 590721769, 2032885395]


Step: 0 ___________________________________________
val_loss: 178.67015075683594
winner_seed: 1471347403


Step: 1 ___________________________________________
val_loss: 176.20993041992188
winner_seed: 2440018909


Step: 2 ___________________________________________
val_loss: 170.63363647460938
winner_seed: 1488588411


Step: 3 ___________________________________________
val_loss: 153.40553283691406
winner_seed: 3987304016


Step: 4 ___________________________________________
val_loss: 176.68060302734375


Step: 5 ___________________________________________
val_loss: 175.87635803222656


Step: 6 ___________________________________________
val_loss: 158.39614868164062


Step: 7 _____________________

2023-10-14 17:02:44.456455: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 166.9215545654297


Step: 16 ___________________________________________


2023-10-14 17:03:41.218397: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 155.49502563476562


Step: 17 ___________________________________________
val_loss: 183.3710479736328


Step: 18 ___________________________________________
val_loss: 170.7713165283203


Step: 19 ___________________________________________
val_loss: 178.34901428222656


Step: 20 ___________________________________________
val_loss: 178.27053833007812


Step: 21 ___________________________________________
val_loss: 161.68804931640625


Step: 22 ___________________________________________
val_loss: 178.23828125


Step: 23 ___________________________________________
val_loss: 180.76296997070312


Step: 24 ___________________________________________
val_loss: 168.19906616210938


final_seed: 2271740860


In [19]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 26ms/step - loss: 9496.0469 - val_loss: 4841.5464
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 6014.0293 - val_loss: 2318.2837
Epoch 3/10000
5/5 [==============================] - 0s 5ms/step - loss: 3616.0081 - val_loss: 17094.3457
Epoch 4/10000
5/5 [==============================] - 0s 6ms/step - loss: 3526.8030 - val_loss: 1286.4576
Epoch 5/10000
5/5 [==============================] - 0s 6ms/step - loss: 449.3432 - val_loss: 455.7245
Epoch 6/10000
5/5 [==============================] - 0s 6ms/step - loss: 272.1942 - val_loss: 278.2271
Epoch 7/10000
5/5 [==============================] - 0s 5ms/step - loss: 236.2321 - val_loss: 319.1407
Epoch 8/10000
5/5 [==============================] - 0s 5ms/step - loss: 175.7401 - val_loss: 311.2893
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 195.7292 - val_loss: 293.7885
Epoch 10/10000
5/5 [==============================] - 0s 5ms/st

Epoch 80/10000
5/5 [==============================] - 0s 6ms/step - loss: 165.1996 - val_loss: 275.2786
Epoch 81/10000
5/5 [==============================] - 0s 6ms/step - loss: 174.2756 - val_loss: 232.6526
Epoch 82/10000
5/5 [==============================] - 0s 6ms/step - loss: 171.6531 - val_loss: 262.1489
Epoch 83/10000
5/5 [==============================] - 0s 6ms/step - loss: 158.8823 - val_loss: 256.0560
Epoch 84/10000
5/5 [==============================] - 0s 7ms/step - loss: 160.8839 - val_loss: 232.3826
Epoch 85/10000
5/5 [==============================] - 0s 6ms/step - loss: 130.4704 - val_loss: 247.9036
Epoch 86/10000
5/5 [==============================] - 0s 6ms/step - loss: 137.0249 - val_loss: 249.5123
Epoch 87/10000
5/5 [==============================] - 0s 6ms/step - loss: 132.0847 - val_loss: 231.3813
Epoch 88/10000
5/5 [==============================] - 0s 6ms/step - loss: 128.7807 - val_loss: 283.0525
Epoch 89/10000
5/5 [==============================] - 0s 7ms/ste

5/5 [==============================] - 0s 5ms/step - loss: 178.6328 - val_loss: 271.5088
Epoch 159/10000
5/5 [==============================] - 0s 5ms/step - loss: 162.9289 - val_loss: 293.3581
Epoch 160/10000
5/5 [==============================] - 0s 5ms/step - loss: 166.3058 - val_loss: 372.0277
Epoch 161/10000
5/5 [==============================] - 0s 5ms/step - loss: 180.0394 - val_loss: 281.6976
Epoch 162/10000
5/5 [==============================] - 0s 5ms/step - loss: 152.9779 - val_loss: 291.8024
Epoch 163/10000
5/5 [==============================] - 0s 5ms/step - loss: 176.7325 - val_loss: 300.6573
Epoch 164/10000
5/5 [==============================] - 0s 5ms/step - loss: 193.1168 - val_loss: 287.1931
Epoch 165/10000
5/5 [==============================] - 0s 5ms/step - loss: 183.0257 - val_loss: 269.0403
Epoch 166/10000
5/5 [==============================] - 0s 5ms/step - loss: 148.7634 - val_loss: 273.8196
Epoch 167/10000
5/5 [==============================] - 0s 5ms/step - lo

5/5 [==============================] - 0s 6ms/step - loss: 178.5208 - val_loss: 292.6400
Epoch 237/10000
5/5 [==============================] - 0s 5ms/step - loss: 167.3965 - val_loss: 289.7225
Epoch 238/10000
5/5 [==============================] - 0s 5ms/step - loss: 171.1003 - val_loss: 264.1168
Epoch 239/10000
5/5 [==============================] - 0s 5ms/step - loss: 159.3799 - val_loss: 271.5948
Epoch 240/10000
5/5 [==============================] - 0s 5ms/step - loss: 181.3968 - val_loss: 287.5844
Epoch 241/10000
5/5 [==============================] - 0s 5ms/step - loss: 142.5347 - val_loss: 265.2107
Epoch 242/10000
5/5 [==============================] - 0s 5ms/step - loss: 153.4660 - val_loss: 266.9742
Epoch 243/10000
5/5 [==============================] - 0s 5ms/step - loss: 155.1306 - val_loss: 251.3108
Epoch 244/10000
5/5 [==============================] - 0s 5ms/step - loss: 165.5310 - val_loss: 287.1119
Epoch 245/10000
5/5 [==============================] - 0s 5ms/step - lo

5/5 [==============================] - 0s 5ms/step - loss: 155.5233 - val_loss: 292.1859
Epoch 315/10000
5/5 [==============================] - 0s 5ms/step - loss: 148.7522 - val_loss: 308.1711
Epoch 316/10000
5/5 [==============================] - 0s 5ms/step - loss: 160.5361 - val_loss: 267.3388
Epoch 317/10000
5/5 [==============================] - 0s 5ms/step - loss: 145.5762 - val_loss: 271.1253
Epoch 318/10000
5/5 [==============================] - 0s 5ms/step - loss: 136.3161 - val_loss: 231.1837
Epoch 319/10000
5/5 [==============================] - 0s 5ms/step - loss: 134.7847 - val_loss: 264.8770
Epoch 320/10000
5/5 [==============================] - 0s 5ms/step - loss: 130.2569 - val_loss: 252.0804
Epoch 321/10000
5/5 [==============================] - 0s 5ms/step - loss: 144.8984 - val_loss: 249.8810
Epoch 322/10000
5/5 [==============================] - 0s 5ms/step - loss: 139.6983 - val_loss: 275.0040
Epoch 323/10000
5/5 [==============================] - 0s 5ms/step - lo

...

5/5 [==============================] - 0s 4ms/step - loss: 114.3680 - val_loss: 244.0245
Epoch 1864/10000
5/5 [==============================] - 0s 5ms/step - loss: 117.5784 - val_loss: 279.4264
Epoch 1865/10000
5/5 [==============================] - 0s 5ms/step - loss: 131.9192 - val_loss: 212.7171
Epoch 1866/10000
5/5 [==============================] - 0s 5ms/step - loss: 114.4651 - val_loss: 220.8180
Epoch 1867/10000
5/5 [==============================] - 0s 5ms/step - loss: 108.4000 - val_loss: 190.0030
Epoch 1868/10000
5/5 [==============================] - 0s 4ms/step - loss: 140.7598 - val_loss: 194.1040
Epoch 1869/10000
5/5 [==============================] - 0s 5ms/step - loss: 111.1647 - val_loss: 205.8408
Epoch 1870/10000
5/5 [==============================] - 0s 4ms/step - loss: 129.5554 - val_loss: 192.6019
Epoch 1871/10000
5/5 [==============================] - 0s 4ms/step - loss: 120.9338 - val_loss: 205.6940
Epoch 1872/10000
5/5 [==============================] - 0s 5ms/

5/5 [==============================] - 0s 4ms/step - loss: 142.4092 - val_loss: 233.0861
Epoch 1941/10000
5/5 [==============================] - 0s 4ms/step - loss: 140.2308 - val_loss: 201.5004
Epoch 1942/10000
5/5 [==============================] - 0s 4ms/step - loss: 132.1343 - val_loss: 206.0983
Epoch 1943/10000
5/5 [==============================] - 0s 4ms/step - loss: 137.4866 - val_loss: 216.4208
Epoch 1944/10000
5/5 [==============================] - 0s 4ms/step - loss: 138.6290 - val_loss: 226.6030
Epoch 1945/10000
5/5 [==============================] - 0s 4ms/step - loss: 136.3846 - val_loss: 224.4317
Epoch 1946/10000
5/5 [==============================] - 0s 4ms/step - loss: 134.3242 - val_loss: 219.4970
Epoch 1947/10000
5/5 [==============================] - 0s 4ms/step - loss: 140.8615 - val_loss: 239.5350
Epoch 1948/10000
5/5 [==============================] - 0s 4ms/step - loss: 150.2887 - val_loss: 239.9437
Epoch 1949/10000
5/5 [==============================] - 0s 4ms/

5/5 [==============================] - 0s 4ms/step - loss: 152.0605 - val_loss: 238.2729
Epoch 2018/10000
5/5 [==============================] - 0s 5ms/step - loss: 133.9650 - val_loss: 221.2791
Epoch 2019/10000
5/5 [==============================] - 0s 5ms/step - loss: 159.7175 - val_loss: 256.5159
Epoch 2020/10000
5/5 [==============================] - 0s 4ms/step - loss: 156.6340 - val_loss: 210.8929
Epoch 2021/10000
5/5 [==============================] - 0s 5ms/step - loss: 135.8855 - val_loss: 196.3543
Epoch 2022/10000
5/5 [==============================] - 0s 4ms/step - loss: 144.0068 - val_loss: 217.1553
Epoch 2023/10000
5/5 [==============================] - 0s 5ms/step - loss: 133.0938 - val_loss: 212.0319
Epoch 2024/10000
5/5 [==============================] - 0s 4ms/step - loss: 147.2570 - val_loss: 197.7360
Epoch 2025/10000
5/5 [==============================] - 0s 4ms/step - loss: 131.5325 - val_loss: 195.0816
Epoch 2026/10000
5/5 [==============================] - 0s 4ms/

5/5 [==============================] - 0s 4ms/step - loss: 138.5736 - val_loss: 240.2544
Epoch 2095/10000
5/5 [==============================] - 0s 4ms/step - loss: 132.7575 - val_loss: 227.4369
Epoch 2096/10000
5/5 [==============================] - 0s 4ms/step - loss: 118.6425 - val_loss: 236.1830
Epoch 2097/10000
5/5 [==============================] - 0s 4ms/step - loss: 131.0344 - val_loss: 257.2813
Epoch 2098/10000
5/5 [==============================] - 0s 4ms/step - loss: 116.5298 - val_loss: 263.7745
Epoch 2099/10000
5/5 [==============================] - 0s 4ms/step - loss: 127.5142 - val_loss: 263.9688
Epoch 2100/10000
5/5 [==============================] - 0s 4ms/step - loss: 131.5040 - val_loss: 193.2367
Epoch 2101/10000
5/5 [==============================] - 0s 4ms/step - loss: 135.2037 - val_loss: 200.8711
Epoch 2102/10000
5/5 [==============================] - 0s 4ms/step - loss: 128.0288 - val_loss: 224.7661
Epoch 2103/10000
5/5 [==============================] - 0s 4ms/

5/5 [==============================] - 0s 4ms/step - loss: 136.4766 - val_loss: 231.9762
Epoch 2172/10000
5/5 [==============================] - 0s 4ms/step - loss: 130.8967 - val_loss: 327.7888
Epoch 2173/10000
5/5 [==============================] - 0s 4ms/step - loss: 138.4400 - val_loss: 227.1339
Epoch 2174/10000
5/5 [==============================] - 0s 4ms/step - loss: 123.4974 - val_loss: 219.8332
Epoch 2175/10000
5/5 [==============================] - 0s 4ms/step - loss: 124.2848 - val_loss: 221.7303
Epoch 2176/10000
5/5 [==============================] - 0s 4ms/step - loss: 128.2433 - val_loss: 205.8339
Epoch 2177/10000
5/5 [==============================] - 0s 4ms/step - loss: 129.4200 - val_loss: 234.7255
Epoch 2178/10000
5/5 [==============================] - 0s 4ms/step - loss: 138.3193 - val_loss: 203.0476
Epoch 2179/10000
5/5 [==============================] - 0s 4ms/step - loss: 133.5505 - val_loss: 282.9898
Epoch 2180/10000
5/5 [==============================] - 0s 4ms/

In [20]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [21]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,80.227158,80.227127,80.227119,80.227097,80.226868,80.190414,-7.633356,12.568946,1.769142,-3.343336,-9.435259,76.773445,-9.435259,-9.43527,-5.886478,6.854057,58.247925,-9.435265,-14.583765,-9.443626,-9.483088,-12.050098,-12.632078,-14.269102,-9.436975,-9.435259,-9.435259,-9.435364,10.775148,-15.318659
Target,88.78,86.195,84.121,88.187,79.514,74.061,64.818,44.254,52.169,53.399,45.364,67.103,61.062,73.445,69.471,83.004,77.332,58.591,70.66,50.796,47.834,45.327,54.032,60.657,67.25,70.773,67.01,81.128,81.737,76.841
Error,8.552841,5.967873,3.893883,7.9599,0.712868,6.129417,72.451355,31.685055,50.399857,56.742332,54.799255,9.670448,70.497261,82.880272,75.357483,76.14994,19.084076,68.02626,85.243767,60.239628,57.317085,57.377098,66.664078,74.926102,76.686974,80.20826,76.445259,90.563362,70.961853,92.15966


In [22]:
display(mae)
display(mape)

52.991787

0.840836

In [23]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [24]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[552.0254]] - Target[827.965]| =  Error: [[275.93964]]; MAPE:[[0.3332745]]
1/1 [==============================] - 0s 12ms/step
Ano-0: |Prediction[[-41.552048]] - Target[752.2110000000002]| =  Error: [[793.76306]]; MAPE:[[1.0552399]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[-42.28637]] - Target[444.73900000000003]| =  Error: [[487.0254]]; MAPE:[[1.0950813]]


[array([[275.93964]], dtype=float32),
 array([[793.76306]], dtype=float32),
 array([[487.0254]], dtype=float32)]

518.90936

0.8278653